<a href="https://colab.research.google.com/github/seoho0529/TensorFlow/blob/main/tf26rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 문자열 토큰 처리 후 LSTM으로 감성분류
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
from keras.models import Sequential
from keras.layers import SimpleRNN, LSTM, Dense

In [12]:
samples = ['The cat say on the mat.','The dog ate my homework']

token_index = {}
for sam in samples:
  for word in sam.split(sep=' '):
    if word not in token_index:
      token_index[word] = len(token_index)
print(token_index)
print()
tokenizer = Tokenizer(num_words=10)
tokenizer.fit_on_texts(samples)
token_seq = tokenizer.texts_to_sequences(samples) # 텍스트를 정수 인덱싱해서 리스트로 반환  # [[1, 2, 3, 4, 1, 5], [1, 6, 7, 8, 9]]
print(token_seq)
print(tokenizer.word_index)
print()
token_mat = tokenizer.texts_to_matrix(samples, mode='binary')  # 'count(빈도)', 'tdidf(비율)', 'freq'등을 쓸 수 있다.
print(token_mat)  # binary를 사용함으로써 원핫 이진벡터 형태로 출력됨
print(tokenizer.word_counts)  # 단어 개수 세기
print(tokenizer.document_count) #문장 개수
print(tokenizer.word_docs)

from keras.utils import to_categorical
token_seq = to_categorical(token_seq[0], num_classes=6)
print(token_seq)

{'The': 0, 'cat': 1, 'say': 2, 'on': 3, 'the': 4, 'mat.': 5, 'dog': 6, 'ate': 7, 'my': 8, 'homework': 9}

[[1, 2, 3, 4, 1, 5], [1, 6, 7, 8, 9]]
{'the': 1, 'cat': 2, 'say': 3, 'on': 4, 'mat': 5, 'dog': 6, 'ate': 7, 'my': 8, 'homework': 9}

[[0. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 1. 1. 1.]]
OrderedDict([('the', 3), ('cat', 1), ('say', 1), ('on', 1), ('mat', 1), ('dog', 1), ('ate', 1), ('my', 1), ('homework', 1)])
2
defaultdict(<class 'int'>, {'cat': 1, 'the': 2, 'mat': 1, 'on': 1, 'say': 1, 'homework': 1, 'dog': 1, 'my': 1, 'ate': 1})
[[0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]]


In [27]:
import numpy as np
docs = ['너무 재밌네요','최고에요','참 잘 만든 영화에요','추천하고 싶은 영화입니다','한 번 더 보고 싶어요','글쎄요','별로에요','생각보다 지루하네요','연기가 어색해요','재미없어요']
# label이 없으면 지도학습을 못하기 때문에..
labels=np.array([1,1,1,1,1,0,0,0,0,0])

token = Tokenizer()
token.fit_on_texts(docs)
print(token.word_index)

x = token.texts_to_sequences(docs) # 정수형 타입을 리스트형으로 반환
print('정수 인덱싱된 토큰 결과 : ', x)

# 요소의 개수가 다르면 힘들기 때문에 요소의 개수를 맞춰줘야 하므로..
from keras.utils import pad_sequences # 서로 다른 길이의 데이터를 일정한 크기로 만듦
from keras.layers import Embedding, Flatten
padded_x = pad_sequences(x, 5)  # 크기를 가장 요소가 긴 개수만큼 맞춤
print(padded_x)

word_size = len(token.word_index) + 1 # 임베딩에 입력될 단어수(토큰 수)를 지정 : 가능한 토큰 수는 최대값 + 1을 준다.
model = Sequential()
# Embedding(가능토큰 수(문장 수), 벡터크기(차원), input_length=시퀀스 개수) - 워드 임베딩이란 텍스트 내의 단어들을 밀집 벡터(dense vector)로 만드는 것을 말한다.
# 워드 임베딩 작업을 수행하고 (numbert of samples, embedding word dimensonality, input_length)인 3D 텐서를 리턴한다.
model.add(Embedding(word_size, 8, input_length=5))
model.add(LSTM(units=32, activation='tanh')) # RNN층
# model.add(Flatten())
model.add(Dense(units=32, activation='relu')) # 완전연결층
model.add(Dense(units=1, activation='sigmoid'))
model.summary()

model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])
model.fit(padded_x, labels, epochs=20, verbose=1)
print('eval acc : %.4f'%(model.evaluate(padded_x, labels)[1]))

print('predict : ', np.where(model.predict(padded_x) > 0.5, 1, 0).ravel())
print('real : ', labels)

{'너무': 1, '재밌네요': 2, '최고에요': 3, '참': 4, '잘': 5, '만든': 6, '영화에요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한': 11, '번': 12, '더': 13, '보고': 14, '싶어요': 15, '글쎄요': 16, '별로에요': 17, '생각보다': 18, '지루하네요': 19, '연기가': 20, '어색해요': 21, '재미없어요': 22}
정수 인덱싱된 토큰 결과 :  [[1, 2], [3], [4, 5, 6, 7], [8, 9, 10], [11, 12, 13, 14, 15], [16], [17], [18, 19], [20, 21], [22]]
[[ 0  0  0  1  2]
 [ 0  0  0  0  3]
 [ 0  4  5  6  7]
 [ 0  0  8  9 10]
 [11 12 13 14 15]
 [ 0  0  0  0 16]
 [ 0  0  0  0 17]
 [ 0  0  0 18 19]
 [ 0  0  0 20 21]
 [ 0  0  0  0 22]]
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 5, 8)              184       
                                                                 
 lstm_7 (LSTM)               (None, 32)                5248      
                                                                 
 dense_14 (Dense)            (None, 32)            

1/1 [==============================] - 1s 548ms/step - loss: 0.6620 - accuracy: 0.9000
eval acc : 0.9000
1/1 [==============================] - 0s 481ms/step
predict :  [1 0 1 1 1 0 0 0 0 0]
real :  [1 1 1 1 1 0 0 0 0 0]
